**TRANSFORMACIONES - files/World_Happiness_Report.csv**

*SIN IMPUTAR - solo nuevas columnas de % y binario (yes/no)*

__________

**Where is this data set from?**

- The World Happiness Report is an annual publication of the United Nations Sustainable Development Solutions Network. This dataset is a subset of the larger report, which includes data from various sources such as the Gallup World Poll and other national surveys. The data was extracted from the World Happiness Report and made available for public use. However, the original data was collected by various researchers and organizations as part of their ongoing efforts to measure and understand happiness and well-being around the world.

    We use observed data on the six variables and estimates of their associations with life evaluations to explain the variation across countries. They include GDP per capita, social support, healthy life expectancy, freedom, generosity, and corruption. Our happiness rankings are not based on any index of these six factors – the scores are instead based on individuals’ own assessments of their lives, in particular, their answers to the single-item Cantril ladder life-evaluation question, much as epidemiologists estimate the extent to which life expectancy is affected by factors such as smoking, exercise, and diet

Detailed information about each of the Predictors:

1. **Log GDP per capita** is in terms of Purchasing Power Parity (PPP) adjusted to a constant 2017 international dollars, taken from the World Development Indicators (WDI) by the World Bank (version 17, metadata last updated on January 22, 2023). See Statistical Appendix 1 for more details. GDP data for 2022 are not yet available, so we extend the GDP time series from 2021 to 2022 using country-specific forecasts of real GDP growth from the OECD Economic Outlook No. 112 (November 2022) or, if missing, from the World Bank’s Global Economic Prospects (last updated: January 10, 2023), after adjustment for population growth. The equation uses the natural log of GDP per capita, as this form fits the data significantly better than GDP per capita.

2. The time series for **Healthy life expectancy at birth** is constructed based on data from the World Health Organization (WHO) Global Health Observatory data repository, with data available for 2005, 2010, 2015, 2016, and 2019. To match this report’s sample period (2005-2022), interpolation and extrapolation are used. See Statistical Appendix 1 for more details.

3. **Social support** - *Conversion: % y yes/no*

    **Social support** (0-1) is the national average of the binary responses (0=no, 1=yes) to the Gallup World Poll (GWP) question “If you were in trouble, do you have relatives or friends you can count on to help you whenever you need them, or not?”

4.  **Freedom to make life choices** - *Conversion: % y yes/no* 

    **Freedom to make life choices** (0-1) is the national average of binary responses to the GWP question “Are you satisfied or dissatisfied with your freedom to choose what you do with your life?”

5. **Generosity** is the residual of regressing the national average of GWP responses to the donation question “Have you donated money to a charity in the past month?” on log GDP per capita.

6.  **Perceptions of corruption** - *Conversion: % y yes/no*  

    **Perceptions of corruption** (0-1) are the average of binary answers to two GWP questions: “Is corruption widespread throughout the government or not?” and “Is corruption widespread within businesses or not?” Where data for government corruption are missing, the perception of business corruption is used as the overall corruption perception measure.

7. **Positive affect** - *Conversion: % y yes/no* 
**Positive affect** is defined as the average of previous-day effects measures for laughter, enjoyment, and interest. The inclusion of interest (first added for World Happiness Report 2022), gives us three components in each of positive and negative affect, and slightly improves the equation fit in column 4. The general form for the affect questions is: Did you experience the following feelings during a lot of the day yesterday?

8. **Negative affect** - *Conversion: % y yes/no* 
**Negative affect** is defined as the average of previous-day effects measures for worry, sadness, and anger.

9. **Life ladder**: Life evaluations from the Gallup World Poll provide the basis for the annual happiness rankings. They are based on answers to the main life evaluation question. The Cantril ladder asks respondents to think of a ladder, with the **best possible life for them being a 10 and the worst possible life being a 0**. They are then asked to rate their own current lives **on a 0 to 10 scale**. The rankings are from nationally representative samples over three years.

10. **Confidence in National Government**: The "Confidence in National Government" variable in the World Happiness Report is calculated based on the following question asked in the Gallup World Poll:

    "Do you have confidence in the national government?"

    Respondents are given the following options to choose from:

    - "Yes, always"
    - "Yes, sometimes"
    - "No, rarely"
    - "No, never"
    - "Don't know"

    **The variable is calculated as the percentage of respondents who answer "Yes, always" or "Yes, sometimes" to this question.**

    This variable is one of several social factors that are included in the calculation of the World Happiness Report's overall happiness score for each country. The report combines data on social factors such as income, social support, life expectancy, freedom to make life choices, generosity, and perceptions of corruption to arrive at a comprehensive measure of happiness.

In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# ------------------------------------------------------------------------------
from scipy.stats import shapiro, kstest

# Scripts de soporte
# -------------------------------------------------
from src.tolookandcompare import to_doc_info, to_doc_headtail, transform_info, transform_headtail

from src import soporte_eda as sp_eda
from src.soporte_eda import resumen_df

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames


**Leer/subir el .csv**

**TRANSFORMACION de `Country Name` al subir el .csv**

In [2]:
# Solucion para cargar .csv al df forzando la conversión del dato a "object"
# df = pd.read_csv ('files/World_Happiness_Report.csv') - original entraba con combo de float, object, nan, etc. de `Country Name``
df = pd.read_csv('files/World_Happiness_Report.csv', dtype={'Country Name': 'object'})

df.sample(5)

,Country Name,Regional Indicator,Year,Life Ladder,Log GDP Per Capita,Social Support,Healthy Life Expectancy At Birth,Freedom To Make Life Choices,Generosity,Perceptions Of Corruption,Positive Affect,Negative Affect,Confidence In National Government
678,Georgia,Commonwealth of Independent States,2013,4.348921,9.370765,0.559166,64.000000,0.722128,-0.257167,0.348714,0.467082,0.199907,0.633563
1421,Nicaragua,Latin America and Caribbean,2015,5.924113,8.623894,0.826909,65.199997,0.809259,0.075562,0.727998,0.771063,0.345595,0.603455
1826,Sri Lanka,South Asia,2020,4.778489,9.480043,0.842500,67.099998,0.803485,-0.048394,0.768454,0.758137,0.285134,0.752757
1602,Qatar,NaN,2015,6.374529,11.532454,NaN,66.400002,NaN,NaN,NaN,NaN,NaN,NaN
1999,Turkmenistan,Commonwealth of Independent States,2013,5.391763,9.312131,0.845733,61.060001,0.704529,-0.071969,NaN,0.551943,0.159606,NaN


In [3]:
df.shape

(2199, 13)

In [4]:
df['Country Name'].dtype

dtype('O')

In [5]:
# NULOS - 0
df['Country Name'].isna().sum()

np.int64(0)

In [6]:
df['Country Name'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina',
       'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain',
       'Bangladesh', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon',
       'Canada', 'Central African Republic', 'Chad', 'Chile', 'China',
       'Colombia', 'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)',
       'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark',
       'Djibouti', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Estonia', 'Eswatini', 'Ethiopia', 'Finland',
       'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana',
       'Greece', 'Guatemala', 'Guinea', 'Guyana', 'Haiti', 'Honduras',
       'Hong Kong S.A.R. of China', 'Hungary', 'Iceland', 'India',
       'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy',
       'Ivory Coast', 'Jamaica', 'Japan', 

In [7]:
df['Country Name'].value_counts()

Country Name
Argentina     17
Costa Rica    17
Brazil        17
Bolivia       17
Bangladesh    17
              ..
Cuba           1
Maldives       1
Guyana         1
Oman           1
Suriname       1
Name: count, Length: 165, dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2199 entries, 0 to 2198
Data columns (total 13 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Country Name                       2199 non-null   object 
 1   Regional Indicator                 2087 non-null   object 
 2   Year                               2199 non-null   int64  
 3   Life Ladder                        2199 non-null   float64
 4   Log GDP Per Capita                 2179 non-null   float64
 5   Social Support                     2186 non-null   float64
 6   Healthy Life Expectancy At Birth   2145 non-null   float64
 7   Freedom To Make Life Choices       2166 non-null   float64
 8   Generosity                         2126 non-null   float64
 9   Perceptions Of Corruption          2083 non-null   float64
 10  Positive Affect                    2175 non-null   float64
 11  Negative Affect                    2183 non-null   float

_________________________

**TRANSFORMACION de las Columnas (5):**

- `Social Support`
- `Freedom To Make Life Choices`
- `Perceptions Of Corruption`
- `Positive Affect`
- `Negative Affect`

Columnas Nuevas: 
- valor numerico decimal en % *(% de si)* 
- valor categorico en binario *(No = hasta .5, Yes = .5+)*, NaN categorico a *"Data not available"*
- sin imputar nulos

In [9]:
# Lista de columnas originales que queremos transformar
cols = [
    "Social Support",
    "Freedom To Make Life Choices",
    "Perceptions Of Corruption",
    "Positive Affect",
    "Negative Affect"
]

for col in cols:
    print(f"\n🔎 Procesando columna: {col}")

    # 1. Contar nulos antes de transformar
    n_nulos = df[col].isna().sum()
    print(f"   ➤ Nulos encontrados: {n_nulos}")

    # 2. Convertir el nombre a Title Case para consistencia en las columnas nuevas
    col_title = col.title()

    # 3. Crear columna en formato porcentaje
    df[f"{col_title} (%)"] = (df[col] * 100).round(1)
    print(f"   ✔ Columna creada: '{col_title} (%)'")

    # 4. Crear columna categórica Yes/No
    df[f"{col_title} (Yes/No)"] = df[col].apply(
        lambda x: "Yes" if pd.notnull(x) and x >= 0.5 else
                  "No" if pd.notnull(x) else
                  "Data not available"
    )
    print(f"   ✔ Columna creada: '{col_title} (Yes/No)'")

    # 5. Mostrar ejemplo rápido de los primeros valores transformados
    print(df[[col, f"{col_title} (%)", f"{col_title} (Yes/No)"]].head(3))




🔎 Procesando columna: Social Support
   ➤ Nulos encontrados: 13
   ✔ Columna creada: 'Social Support (%)'
   ✔ Columna creada: 'Social Support (Yes/No)'
   Social Support  Social Support (%) Social Support (Yes/No)
0        0.450662                45.1                      No
1        0.552308                55.2                     Yes
2        0.539075                53.9                     Yes

🔎 Procesando columna: Freedom To Make Life Choices
   ➤ Nulos encontrados: 33
   ✔ Columna creada: 'Freedom To Make Life Choices (%)'
   ✔ Columna creada: 'Freedom To Make Life Choices (Yes/No)'
   Freedom To Make Life Choices  Freedom To Make Life Choices (%)  \
0                      0.718114                              71.8   
1                      0.678896                              67.9   
2                      0.600127                              60.0   

  Freedom To Make Life Choices (Yes/No)  
0                                   Yes  
1                                   Yes  

*NaNs % se mantienen porque Tableau/Power BI pueden graficarla sin errores.*

In [10]:
df.sample(5)

,Country Name,Regional Indicator,Year,Life Ladder,Log GDP Per Capita,Social Support,Healthy Life Expectancy At Birth,Freedom To Make Life Choices,Generosity,Perceptions Of Corruption,Positive Affect,Negative Affect,Confidence In National Government,Social Support (%),Social Support (Yes/No),Freedom To Make Life Choices (%),Freedom To Make Life Choices (Yes/No),Perceptions Of Corruption (%),Perceptions Of Corruption (Yes/No),Positive Affect (%),Positive Affect (Yes/No),Negative Affect (%),Negative Affect (Yes/No)
1818,Sri Lanka,South Asia,2011,4.180569,9.229197,0.841938,64.680000,0.822637,0.139286,0.760301,0.730496,0.174927,0.855967,84.2,Yes,82.3,Yes,76.0,Yes,73.0,Yes,17.5,No
668,Gambia,Sub-Saharan Africa,2018,4.922099,7.607178,0.684800,56.700001,0.718729,0.443072,0.691070,0.758668,0.379208,0.757543,68.5,Yes,71.9,Yes,69.1,Yes,75.9,Yes,37.9,No
1359,Namibia,Sub-Saharan Africa,2019,4.435811,9.211298,0.844592,56.099998,0.739035,-0.179359,0.879071,0.643717,0.256161,0.559564,84.5,Yes,73.9,Yes,87.9,Yes,64.4,Yes,25.6,No
1110,Lebanon,Middle East and North Africa,2015,5.171971,9.717864,0.741708,65.699997,0.596750,0.067901,0.888953,0.524465,0.242554,0.187712,74.2,Yes,59.7,Yes,88.9,Yes,52.4,Yes,24.3,No
1497,Pakistan,South Asia,2017,5.830871,8.495299,0.690264,56.349998,0.712657,0.038074,0.713928,0.488859,0.308341,0.557435,69.0,Yes,71.3,Yes,71.4,Yes,48.9,No,30.8,No


____

#### **GUARDAR nuevo .csv "World Happiness Report limpio sin imputar":** 

In [11]:
df.to_csv('files/World_Happiness_Report_limpio_sin_imputar.csv', index=False)

In [12]:
df.head(5)

,Country Name,Regional Indicator,Year,Life Ladder,Log GDP Per Capita,Social Support,Healthy Life Expectancy At Birth,Freedom To Make Life Choices,Generosity,Perceptions Of Corruption,Positive Affect,Negative Affect,Confidence In National Government,Social Support (%),Social Support (Yes/No),Freedom To Make Life Choices (%),Freedom To Make Life Choices (Yes/No),Perceptions Of Corruption (%),Perceptions Of Corruption (Yes/No),Positive Affect (%),Positive Affect (Yes/No),Negative Affect (%),Negative Affect (Yes/No)
0,Afghanistan,South Asia,2008,3.723590,7.350416,0.450662,50.500000,0.718114,0.167652,0.881686,0.414297,0.258195,0.612072,45.1,No,71.8,Yes,88.2,Yes,41.4,No,25.8,No
1,Afghanistan,South Asia,2009,4.401778,7.508646,0.552308,50.799999,0.678896,0.190809,0.850035,0.481421,0.237092,0.611545,55.2,Yes,67.9,Yes,85.0,Yes,48.1,No,23.7,No
2,Afghanistan,South Asia,2010,4.758381,7.613900,0.539075,51.099998,0.600127,0.121316,0.706766,0.516907,0.275324,0.299357,53.9,Yes,60.0,Yes,70.7,Yes,51.7,Yes,27.5,No
3,Afghanistan,South Asia,2011,3.831719,7.581259,0.521104,51.400002,0.495901,0.163571,0.731109,0.479835,0.267175,0.307386,52.1,Yes,49.6,No,73.1,Yes,48.0,No,26.7,No
4,Afghanistan,South Asia,2012,3.782938,7.660506,0.520637,51.700001,0.530935,0.237588,0.775620,0.613513,0.267919,0.435440,52.1,Yes,53.1,Yes,77.6,Yes,61.4,Yes,26.8,No
